<a href="https://colab.research.google.com/github/casualcomputer/llm_google_colab/blob/main/llm_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM using Gemini (langchain)

In [68]:
!python -V

Python 3.11.12


In [55]:
# Install dependencies
!pip install --upgrade pip --quiet
!pip install langchain google-generativeai torch torchvision transformers datasets langchain_community langchain_google_genai--quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement langchain_google_genai--quiet (from versions: none)
ERROR: No matching distribution found for langchain_google_genai--quiet


In [56]:
# Imports & Authentication
import os
from datasets import load_dataset
import re
import pandas as pd
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY') # Use the name you gave your secret

if api_key:
    # Use the api_key
    print("API key loaded successfully from Colab Secrets.")
    # print(f"Your API key is: {api_key}") # Be careful printing the key itself
else:
    print("API key 'GOOGLE_API_KEY' not found in Colab Secrets.")

API key loaded successfully from Colab Secrets.


In [57]:
!pip install langchain_google_genai --quiet
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest", # You can also try "gemini-pro"
            temperature=0,
            google_api_key=api_key
        )
print("LLM Initialized successfully.")

print("\nSending a test prompt to the LLM...")
print(type(llm))
print(llm.invoke("Write me a ballad about LangChain").content[:90])

LLM Initialized successfully.

Sending a test prompt to the LLM...
<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>
(Verse 1)
The data streams, a raging flood,
A chaos vast, misunderstood.
To tame its power


# Agent

## Define Serper Tool

In [58]:
!pip install browserbase 'crewai[tools]' --quiet
from crewai_tools import SerperDevTool
import os
from google.colab import userdata

# Load the Serper API key from Colab Secrets
serper_api_key = userdata.get('SERPER_API_KEY') # Use the name you gave your Serper secret

if serper_api_key:
    # Set the SERPER_API_KEY environment variable
    os.environ["SERPER_API_KEY"] = serper_api_key
    print("SERPER_API_KEY loaded and set successfully from Colab Secrets.")
else:
    print("SERPER_API_KEY not found in Colab Secrets.")


# Initialize the tool for internet searching capabilities
# Now the tool will find the SERPER_API_KEY in the environment variables
Serpertool = SerperDevTool(
    search_url="https://google.serper.dev/scholar",
    n_results=10,
)

# You can now run the tool
#print(Serpertool.run(search_query="ChatGPT"))

SERPER_API_KEY loaded and set successfully from Colab Secrets.


## Create a CrewAI agent

## Create prompt templates for agent switching

In [60]:
# 3. Define all your prompt templates from Section 1
ZERO_SHOT_TEMPLATE = "You are a helpful assistant. Answer concisely:\n\n{query}"
FEW_SHOT_TEMPLATE = (
    "You are a helpful assistant. Use the examples below to guide your answer.\n\n"
    "Example 1:\nQ: Who wrote 'Pride and Prejudice'?\nA: Jane Austen\n\n"
    "Example 2:\nQ: What is the boiling point of water in Celsius?\nA: 100°C\n\n"
    "Now answer:\nQ: {query}\nA:"
)
COT_TEMPLATE = (
    "You are a reasoning assistant. Think step by step:\n\n"
    "Question: {query}\n"
    "Answer: Let’s think step by step."
)
COT_SC_TEMPLATE = (
    "You are a reasoning assistant. Think carefully, step by step:\n\n"
    "Question: {query}\n"
    "Answer:"
)
REACT_TEMPLATE = (
    "You are a multi-agent reasoning assistant. Follow the ReAct format exactly:\n\n"
    "Thought: I need to gather facts for: {query}\n"
    "Action: Search[\"{query}\"]\n"
    "Observation:\n"
    "Thought: Now I have the necessary information.\n"
    "Answer:"
)
RAG_TEMPLATE = (
    "You have been provided the following context passages:\n\n{retrieved_docs}\n\n"
    "Using only the above context, answer:\n\n{query}"
)
CHAIN_STEP1 = (
    "Extract key facts needed to answer:\n\n"
    "Question: {query}\n"
    "Facts:"
)
CHAIN_STEP2 = (
    "You have these facts:\n\n{facts}\n\n"
    "Now answer:\n\n"
    "Question: {query}\n"
    "Answer:"
)
META_PROMPT = (
    "You are a prompt engineer. Given the user’s instruction, produce an optimized prompt:\n\n"
    "User instruction: {user_instruction}\n\n"
    "Optimized prompt:"
)
PAL_TEMPLATE = (
    "You are a Python interpreter. Write Python code that solves:\n\n"
    "{query}\n\n"
    "# After writing the code, execute it mentally and provide the result."
)

# 4. Build a dispatcher function to select the right template
def build_prompt(query: str, mode: str, **kwargs) -> str:
    if mode == "zero_shot":
        return ZERO_SHOT_TEMPLATE.format(query=query)
    elif mode == "few_shot":
        return FEW_SHOT_TEMPLATE.format(query=query)
    elif mode == "cot":
        return COT_TEMPLATE.format(query=query)
    elif mode == "cot_sc":
        return COT_SC_TEMPLATE.format(query=query)
    # elif mode == "rag":
    #     return RAG_TEMPLATE.format(retrieved_docs=kwargs.get("retrieved_docs", ""), query=query)
    elif mode == "chain_step1":
        return CHAIN_STEP1.format(query=query)
    elif mode == "chain_step2":
        return CHAIN_STEP2.format(query=query, facts=kwargs.get("facts", ""))
    elif mode == "meta":
        return META_PROMPT.format(user_instruction=query)
    elif mode == "pal":
        return PAL_TEMPLATE.format(query=query)
    else:
        raise ValueError(f"Unknown mode: {mode}")


In [61]:
# 0. a list to hold them
raw_steps: list = []

def collect_raw(obj, **_):
    """
    CrewAI passes AgentAction / ToolResult / AgentFinish
    (or sometimes a plain str).  Keep each object intact.
    """
    import copy                     # avoid later mutation
    raw_steps.append(copy.deepcopy(obj))

# 1) Re‐use a “stringify” helper that knows how to format each step
def stringify_step(obj, max_obs_chars: int = 600) -> str:
    # If it arrives as a bare string, that’s the final answer
    if isinstance(obj, str):
        return f"Final Answer: {obj.strip()}"
    # If it’s an AgentAction, print thought / tool name / tool_input
    if hasattr(obj, "thought") and hasattr(obj, "tool_input"):
        return (
            f"Thought: {obj.thought.strip()}\n"
            f"Action: {obj.tool}\n"
            f"Action Input: {obj.tool_input}"
        )
    # If it’s a ToolResult, include the preceding thought (in .text or .log) and the result
    # (some versions store the prior thought in .text or .log, adjust accordingly)
    if hasattr(obj, "result"):
        # many builds put the prior “thought” text into obj.text
        prior = getattr(obj, "text", "").strip() or ""
        obs   = obj.result
        if len(str(obs)) > max_obs_chars:
            obs = str(obs)[:max_obs_chars] + " …[truncated]…"
        if prior:
            return f"{prior}\nObservation: {obs}"
        else:
            return f"Observation: {obs}"
    # If it’s an AgentFinish, the final answer lives in obj.text
    if hasattr(obj, "text"):
        return f"Final Answer: {obj.text.strip()}"
    # Fallback to repr()
    return repr(obj)

In [45]:
raw_steps

[ToolResult(result="{'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourhoods in...', 'link': 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Ottawa'}, {'title': 'Timeline of Ottawa history', 'link': 'https://en.wikipedia.org/wiki/Timeline_of_Ottawa_history'}]}, {'title': 'Demographics of Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa', 'snippet': 'In 2021, the population of the city of Ottawa was 1,017,44

In [53]:
triple = raw_steps      # the list you showed
print(raw_steps)

tool_res   = triple[0]          # ToolResult
action_obj = triple[1]          # AgentAction
finish_obj = triple[2]          # AgentFinish

print(dir(action_obj))        # or:  pprint(vars(action_obj))

# 1) Print the LLM’s “thought” string:
print("Thought:\n", action_obj.thought)

# 2) Print which tool was invoked (e.g. "Search"):
print("Action :", action_obj.tool)

# 3) Print the payload sent to that tool:
print("Input  :", action_obj.tool_input)

print("Action via tool use:", tool_res)

print("Final Answer:\n", finish_obj.text)

# # 2) Convert every raw step into a string
# text_lines = [stringify_step(o) for o in raw_steps]

# # 3) Now either print them or write to disk. For example:
# with open("trace_log.txt", "w", encoding="utf-8") as f:
#     for line in text_lines:
#         f.write(line.replace("\n", "\n    ") + "\n\n")  # indent continuations

# # 4) If you just want a single big string:
# full_text = "\n\n".join(text_lines)
# print(full_text)

[ToolResult(result="{'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourhoods in...', 'link': 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Ottawa'}, {'title': 'Timeline of Ottawa history', 'link': 'https://en.wikipedia.org/wiki/Timeline_of_Ottawa_history'}]}, {'title': 'Demographics of Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa', 'snippet': 'In 2021, the population of the city of Ottawa was 1,017,44

In [64]:
!pip install crewai crewai_tools --quiet
from crewai import LLM, Agent, Task, Crew
import os

# ---- 1. LLM wrapper -------------------------------------------------
llm_crew = LLM(
    model="gemini/gemini-1.5-flash-latest",       # provider prefix
    api_key=api_key          # set GOOGLE_API_KEY env var
)

# ---- 3. ReAct agent --------------------------------------------------
agent_react = Agent(
    role="Factual Researcher",
    goal="Find accurate facts quickly",
    backstory=(
        "You are a detail-oriented research analyst who uses web search "
        "to answer geography and demographic questions with concise, up-to-date data."
    ),
    llm=llm_crew,
    tools=[Serpertool],
    step_callback= collect_raw, # collect_step,   # register our function to receive every step
    verbose=True          # prints Thought / Action / Observation
)

# ---- 4. One-off task runner -----------------------------------------
!pip install rich
from crewai import Task, Crew    # already imported Agent earlier
from rich import get_console

def run_single_agent(query: str) -> str:
    #step_trace.clear()              # reset collector
    task = Task(
        description=query,
        expected_output="Concise answer",
        agent=agent_react
    )
    crew = Crew(agents=[agent_react], tasks=[task])
    result = crew.kickoff_for_each(inputs=[{"input": query}])[0].raw

    # Normalise return type
    if isinstance(result, str):
        return result.strip()
    if isinstance(result, dict):
        return result
    return str(result).strip()

## Make LLM calls

In [69]:
from langchain_core.messages import BaseMessage   # for the isinstance check

def _to_text(resp) -> str:
    """Normalise CrewAI responses to a clean string."""
    if isinstance(resp, str):
        return resp
    if isinstance(resp, BaseMessage):
        return resp.content
    return str(resp)          # fallback

def smart_llm_call(model, prompt):
    """
    Call the LLM using whichever entry-point the installed
    CrewAI / LangChain version supports.
    """
    for m in ("invoke", "predict", "__call__"):
        if hasattr(model, m):
            resp = getattr(model, m)(prompt)     # call the method
            return _to_text(resp)

# ------------------ MAIN DISPATCHER ------------------
def call_with_mode(query: str, mode: str, **kwargs) -> str:

    # if mode == "react":          # needs the Search tool
    #   return smart_llm_call(agent_react.llm, prompt).strip()
    if mode == "react":
      return run_single_agent(query)
    else:                        # every other prompting style
      prompt = build_prompt(query, mode, **kwargs)
      return smart_llm_call(llm, prompt).strip()

# 6. Example Calls
query_example = "What is the capital of Canada and its population?"
react_answer = call_with_mode(query_example, "react")
print("react →", react_answer)
print("react log →", [stringify_step(o) for o in raw_steps])

# 6A. Zero-Shot
res_zero = call_with_mode(query_example, mode="zero_shot")
print("Zero-Shot:", res_zero)  # :contentReference[oaicite:24]{index=24}
print("...................")

# 6B. Few-Shot
res_few = call_with_mode(query_example, mode="few_shot")
print("Few-Shot:", res_few)    # :contentReference[oaicite:25]{index=25}
print("...................")

# 6C. Chain-of-Thought (CoT)
res_cot = call_with_mode(query_example, mode="cot")
print("CoT:", res_cot)         # :contentReference[oaicite:26]{index=26}
print("...................")

# 6D. Self-Consistency (CoT-SC) (voting over 3 runs)
cot_sc_chains = [call_with_mode(query_example, mode="cot_sc") for _ in range(3)]
finals = [chain.split("\n")[-1].strip() for chain in cot_sc_chains if "Answer:" in chain]
from collections import Counter
res_cot_sc = Counter(finals).most_common(1)[0][0] if finals else ""
print("CoT-SC Vote:", res_cot_sc)  # :contentReference[oaicite:27]{index=27}
print("...................")

# 6G. Prompt Chaining (Two-Step)
facts = call_with_mode(query_example, mode="chain_step1")
print("Chained Facts:", facts)  # :contentReference[oaicite:30]{index=30}
res_chain_final = call_with_mode(query_example, mode="chain_step2", facts=facts)
print("Chained Final:", res_chain_final)  # :contentReference[oaicite:31]{index=31}
print("...................")

# 6H. Meta Prompting
opt_prompt = call_with_mode("Optimize a prompt for: " + query_example, mode="meta")
print("Meta Optimized Prompt:", opt_prompt)  # :contentReference[oaicite:32]{index=32}
print("...................")

# 6I. Program-Aided Language Model (PAL)
res_pal = call_with_mode(query_example, mode="pal")
print("PAL:", res_pal)         #
print("...................")

# # 6F. Retrieval-Augmented Generation (RAG)
# # Assume retrieve_docs is implemented elsewhere
# def retrieve_docs(q: str, k: int = 3) -> str:
#     # E.g., use a FAISS vector store or Wikipedia API
#     return "Ottawa is the capital of Canada. The population is ~1 million."

# docs = retrieve_docs(query_example, k=3)
# res_rag = call_with_mode(query_example, mode="rag", retrieved_docs=docs)
# print("RAG:", res_rag)         # :contentReference[oaicite:29]{index=29}

# Agent: Factual Researcher
## Task: What is the capital of Canada and its population?











# Agent: Factual Researcher
## Thought: tool_code
Thought: I need to find the capital of Canada and its population using a web search.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"population of Ottawa Canada\"}"
## Tool Output: 
{'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourhoods in...', 'link': 'https://en.w

# Routing Algorithms

In [91]:
# Seed queries with known expected answers (realistic from public datasets)
seed_queries = [
    # 1. GSM8K example (grade‐school math word problem)
    (
        "Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. "
        "If she used a total of 120 teaspoons of sugar and cups of water, how many teaspoons of sugar did she use?",
        "42"
    ),
    # 2. CommonsenseQA (core factual QA)
    (
        "Who is credited with inventing the telephone?",
        "Bell"
    ),
    # 3. CoQA (conversational QA, simple factual)
    (
        "Where is the Eiffel Tower located?",
        "Paris"
    ),
    # 4. HumanEval (code generation – PAL)
    (
        "Write a Python function that returns the nth Fibonacci number.",
        "def"
    ),
    # 5. HotpotQA (multi‐hop QA requiring Wikipedia facts)
    (
        "What is the capital of the country whose flag features a maple leaf?",
        "Ottawa"
    ),
    # 6. SQuAD 2.0 (reading comprehension – factual date)
    (
        "Which year did Alexander Graham Bell receive his patent for the telephone?",
        "1876"
    ),
    # 7. MBPP (basic Python programming task)
    (
        "Complete the function to reverse a list in Python.",
        "def"
    ),
    # 8. SocialIQA (commonsense reasoning about emotions)
    (
        "If someone returned a lost dog to its owner, how would the owner feel?",
        "grateful"
    ),
]

In [93]:
import json
import pandas as pd

labeled_data = []
modes = ["zero_shot", "few_shot", "cot", "pal", "react"]

# 1. Prepare a list to store full outputs
all_mode_outputs = []

for (query, expected) in seed_queries:
    outputs = {}
    for mode in modes:
        try:
            resp = call_with_mode(query, mode)
        except Exception:
            resp = ""
        outputs[mode] = resp

    # Determine best_mode (as before)
    if re.search(re.escape(expected), outputs["zero_shot"], re.IGNORECASE):
        best_mode = "zero_shot"
    elif re.search(re.escape(expected), outputs["few_shot"], re.IGNORECASE):
        best_mode = "few_shot"
    elif re.search(re.escape(expected), outputs["cot"], re.IGNORECASE):
        best_mode = "cot"
    elif re.search(re.escape(expected), outputs["pal"], re.IGNORECASE):
        best_mode = "pal"
    elif re.search(re.escape(expected), outputs["react"], re.IGNORECASE):
        best_mode = "react"
    else:
        best_mode = "zero_shot"

    # 2. Append to labeled_data as before
    labeled_data.append((query, best_mode, expected))

    # 3. Also record every mode’s raw output
    record = {"query": query, "expected": expected, "best_mode": best_mode}
    for mode in modes:
        record[f"{mode}_output"] = outputs[mode]
    all_mode_outputs.append(record)

# 4. Now all_mode_outputs is a list of dicts:
#    [
#      {
#        "query": "...",
#        "expected": "...",
#        "best_mode": "...",
#        "zero_shot_output": "...",
#        "few_shot_output": "...",
#        "cot_output": "...",
#        "pal_output": "...",
#        "react_output": "..."
#      },
#      ...
#    ]


# Agent: Factual Researcher
## Task: Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, how many teaspoons of sugar did she use?











# Agent: Factual Researcher
## Thought: tool_code
Thought:To find the number of teaspoons of sugar Katy used, I need to determine the proportion of sugar in the total mixture.  The ratio of sugar to water is 7:13, meaning there are 7 parts sugar for every 13 parts water, for a total of 7 + 13 = 20 parts.  I can then use this proportion to calculate the amount of sugar used.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"ratio calculation\"}"
## Tool Output: 
{'searchParameters': {'q': 'ratio calculation', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ratio Calculator', 'link': 'https://www.calculatorsoup.com/calculators/math/ratios.php', 'snippet': 'Calculator solves ratios for the missing

ModuleNotFoundError: No module named 'ace_tools'

In [96]:
all_mode_outputs

[{'query': 'Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, how many teaspoons of sugar did she use?',
  'expected': '42',
  'best_mode': 'zero_shot',
  'zero_shot_output': '7/20 * 120 = 42 teaspoons of sugar',
  'few_shot_output': 'Let the ratio of teaspoons of sugar to cups of water be 7:13.\nThe total number of parts in the ratio is 7 + 13 = 20 parts.\nKaty used a total of 120 teaspoons of sugar and cups of water.\nLet x be the number of parts. Then 20x = 120.\nx = 120 / 20 = 6\nThe number of teaspoons of sugar used is 7 parts, which is 7 * 6 = 42 teaspoons.\nThe number of cups of water used is 13 parts, which is 13 * 6 = 78 cups.\nTotal = 42 + 78 = 120.\n\nTherefore, Katy used $\\boxed{42}$ teaspoons of sugar.',
  'cot_output': 'Let’s think step by step.\n\n**Step 1: Understand the ratio.**\n\nThe ratio of sugar to water is 7:13.  This means for every 7 teaspoons of sugar, Katy uses 

In [97]:
# 5. To inspect in a DataFrame:
df_outputs = pd.DataFrame(all_mode_outputs)
df_outputs
#import ace_tools as tools; tools.display_dataframe_to_user(name="All Mode Outputs", dataframe=df_outputs)

,query,expected,best_mode,zero_shot_output,few_shot_output,cot_output,pal_output,react_output
0,Katy makes coffee using teaspoons of sugar and...,42,zero_shot,7/20 * 120 = 42 teaspoons of sugar,Let the ratio of teaspoons of sugar to cups of...,Let’s think step by step.\n\n**Step 1: Underst...,```python\ntotal_parts = 7 + 13 # Total parts...,42
1,Who is credited with inventing the telephone?,Bell,zero_shot,Alexander Graham Bell,Alexander Graham Bell,Let's think step by step.\n\nStep 1: The ques...,"```python\ndef get_telephone_inventor():\n """"...",Alexander Graham Bell is credited with inventi...
2,Where is the Eiffel Tower located?,Paris,zero_shot,"Paris, France.","Paris, France",Let's think step by step.\n\n1. **Identify the...,"```python\nlocation_data = {\n ""Eiffel Towe...","Champ de Mars in Paris, France"
3,Write a Python function that returns the nth F...,def,zero_shot,```python\ndef fibonacci(n):\n if n <= 0:\n ...,"```python\ndef fibonacci(n):\n """"""\n This fu...",Let's think step by step.\n\n**1. Understandin...,"```python\ndef fibonacci(n):\n """"""\n This fu...",```python\ndef fibonacci(n):\n if n <= 0:\n...
4,What is the capital of the country whose flag ...,Ottawa,zero_shot,Ottawa,Ottawa,Let’s think step by step.\n\n1. **Identify the...,I can't access external information like datab...,Ottawa
5,Which year did Alexander Graham Bell receive h...,1876,zero_shot,1876,1876,Let’s think step by step.\n\n1. **Identify the...,I can't directly access external information l...,1876
6,Complete the function to reverse a list in Pyt...,def,zero_shot,```python\ndef reverse_list(lst):\n return ls...,"```python\ndef reverse_list(lst):\n """"""Revers...",Let's think step by step. We need to create a...,"```python\ndef reverse_list(input_list):\n """"...",```python\ndef reverse_list(lst):\n return ls...
7,"If someone returned a lost dog to its owner, h...",grateful,zero_shot,Relieved and grateful.,"The owner would likely feel relieved, happy, a...",Let’s think step step.\n\n1. **The situation:*...,The question is not directly solvable with Pyt...,"Overwhelming relief, joy, and happiness. Many..."


In [98]:
df_outputs.to_csv("all_mode_outputs.csv")

## Create training dataset (define the best thinking mode for each question in training data)

1. **Correctness First**

  For each query, we have a column in all_mode_outputs.csv for each mode’s raw text and a Boolean (via regex) indicating whether that mode “got the expected answer” (i.e. contained the reference token). Any mode that contains the expected keyword is considered correct for that query.

2. **Cost Second**

  Among the modes that are correct, I pick the one with the lowest “cost.” In most LLM‐as‐a‐service pricing models, cost is roughly proportional to the number of tokens sent + received and/or the number of external API calls. In practice:

  * zero_shot (no examples, no extra “chain‐of‐thought” or tool calls) typically uses the fewest tokens → lowest cost.

  * few_shot (you prepend 1–3 examples) uses more tokens than zero_shot but fewer than CoT or React.

  * cot (“Let’s think step by step”) injects extra words per query → higher token count.

  * pal (code → mental execution) often generates a full code snippet plus an explanation → medium‐high token usage.

  * react (ReAct + Serper search) may incur both “extra tokens” in the LLM prompt and an external web‐search API call. It usually ends up the most expensive (because you pay for the search + the resulting LLM tokens).

In [100]:
import re
import pandas as pd

df = pd.read_csv("all_mode_outputs.csv")

def is_correct(row, mode):
    """
    Returns True if the mode’s output contains the expected token (case‐insensitive).
    """
    expected = str(row["expected"])  # reference keyword/phrase
    output = str(row[f"{mode}_output"])
    return bool(re.search(re.escape(expected), output, re.IGNORECASE))

results = []
for idx, row in df.iterrows():
    # Build a list of all modes that got the answer “right”
    correct_modes = []
    for mode in ["zero_shot", "few_shot", "cot", "pal", "react"]:
        if is_correct(row, mode):
            correct_modes.append(mode)

    # If none matched, fallback to zero_shot
    if not correct_modes:
        chosen = "zero_shot"
    else:
        # Otherwise choose among correct_modes by lowest “approximate cost”
        # We’ll assign a relative cost rank (lower = cheaper):
        cost_rank = {
            "zero_shot": 1,
            "few_shot": 2,
            "cot": 3,
            "pal": 3,   # PAL might be slightly cheaper than CoT if code is compact
            "react": 4  # React is most expensive (search + tokens)
        }
        chosen = min(correct_modes, key=lambda m: cost_rank[m])

    results.append({
        "query": row["query"],
        "expected": row["expected"],
        "correct_modes": correct_modes,
        "chosen_mode": chosen
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))


                                                                                                                                                                                       query expected                          correct_modes chosen_mode
Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, how many teaspoons of sugar did she use?       42 [zero_shot, few_shot, cot, pal, react]   zero_shot
                                                                                                                                               Who is credited with inventing the telephone?     Bell [zero_shot, few_shot, cot, pal, react]   zero_shot
                                                                                                                                                          Where is the Eiffel Tower located?    Paris [zero_shot, few_shot, cot, pal, react]   zero_shot
    

## Synthetic Training Data
In a real setting, zero-shot might solve all these queries on a powerful model like Gemini. However, for smaller models or complex tasks, we assume the following:

*   CoT (Chain‐of‐Thought) often outperforms zero-shot on multi-step logic or arithmetic
*   PAL excels when code generation is necessary (e.g., algorithmic or parsing tasks).
*   Few-Shot helps on ambiguous or multiple-choice queries by providing exemplars.
*   ReAct shines on retrieval-heavy or very up-to-date factual questions.

References

- Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., … Amodei, D. (2020). Language Models Are Few-Shot Learners. In NeurIPS. https://arxiv.org/abs/2005.14165

- Wei, J., Wang, X., Schuurmans, D., Le, Q., Elliott, D., Liu, P., & Zhou, D. (2022). Chain-of-Thought Prompting Elicits Reasoning in Large Language Models. In NeurIPS. https://arxiv.org/abs/2201.11903

- Kojima, T., Cai, T., Dou, Z., & Neubig, G. (2022). Large Language Models Are Zero-Shot Reasoners. In ICLR. https://arxiv.org/abs/2205.08522

- Yao, S.-W., Miao, X., Wu, Y., Yu, L., Wang, R., Qu, L., & Liu, J. (2023). ReAct: Synergizing Reasoning and Acting in Language Models. In NeurIPS. https://arxiv.org/abs/2310.02468

- Google DeepMind. (2024). Introducing Gemini: Building Multimodal Agents That Can Reason, Collaborate, and Explain. Google DeepMind Blog. https://blog.deepmind.com/introducing-gemini/

In [101]:
# 1. Synthetic Dataset Definition
data = [
    {"query": "Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 parts, how many teaspoons of sugar did she use?", "expected": "42", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "cot"},
    {"query": "Who is credited with inventing the telephone?", "expected": "Bell", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "Where is the Eiffel Tower located?", "expected": "Paris", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Write a Python function that returns the nth Fibonacci number.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the capital of the country whose flag features a maple leaf?", "expected": "Ottawa", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Which year did Alexander Graham Bell receive his patent for the telephone?", "expected": "1876", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "cot"},
    {"query": "Complete the function to reverse a list in Python.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "If someone returned a lost dog to its owner, how would the owner feel?", "expected": "grateful", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "few_shot"},
    {"query": "Calculate the square root of 1521.", "expected": "39", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Given a CSV file of sales data, write Python code to compute the total revenue.", "expected": "import", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "In a conversation, 'Give me directions to the nearest bakery,' what is the user’s intent?", "expected": "intent", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "few_shot"},
    {"query": "According to the latest NASA press release, what is the name of their newest Mars rover?", "expected": "Perseverance", "correct_modes": ["react"], "chosen_mode": "react"},
    {"query": "Solve for x: 3x + 7 = 22.", "expected": "5", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "cot"},
    {"query": "Translate the following English sentence to French: 'The quick brown fox jumps over the lazy dog.'", "expected": "rapide", "correct_modes": ["zero_shot", "few_shot"], "chosen_mode": "few_shot"},
    {"query": "Which library in Python is most commonly used for data manipulation with DataFrames?", "expected": "pandas", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "How many minutes are there in 3.5 hours?", "expected": "210", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Write a SQL query to select all customers from New York from a 'customers' table.", "expected": "SELECT", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "pal"},
    {"query": "Who painted the Mona Lisa?", "expected": "da Vinci", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Explain why the sky appears red at sunset.", "expected": "Rayleigh", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "cot"},
    {"query": "Compute the factorial of 7 using Python.", "expected": "factorial(7)", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the boiling point of water at sea level in Celsius?", "expected": "100°C", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Who wrote 'Pride and Prejudice'?", "expected": "Austen", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "few_shot"},
    {"query": "What is the GDP of Nigeria in 2020 (in billions)?", "expected": "432", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Solve the integral ∫ (2x³ + 3x²) dx.", "expected": "½x⁴ + x³", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Write a bash command to list all files in the current directory.", "expected": "ls", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "zero_shot"},
    {"query": "Translate 'Good morning, how are you?' to Spanish.", "expected": "Buenos", "correct_modes": ["zero_shot", "few_shot"], "chosen_mode": "few_shot"},
    {"query": "What is the capital city of Australia?", "expected": "Canberra", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Given a list of integers, write Python code to return only the even numbers.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "Which planet in our solar system has the most moons?", "expected": "Jupiter", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "If a car travels at 60 mph, how far will it travel in 2.5 hours?", "expected": "150", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Who is the author of '1984' and in what year was it first published?", "expected": "Orwell", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "Create a JavaScript function to check if a string is a palindrome.", "expected": "function", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the chemical symbol for gold?", "expected": "Au", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "zero_shot"},
    {"query": "Explain the difference between supervised and unsupervised learning.", "expected": "labeled", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "cot"},
    {"query": "Find the shortest path between two nodes in a graph using pseudocode.", "expected": "while", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Who won the FIFA World Cup in 2018?", "expected": "France", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "Write Python code to generate a random integer between 1 and 100.", "expected": "import", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "pal"},
    {"query": "In Shakespeare's 'Hamlet,' who says 'To be, or not to be'?", "expected": "Hamlet", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "Solve for y: 2y² – 8y + 6 = 0.", "expected": "y=1 or y=3", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"}
]

df = pd.DataFrame(data)
df

,query,expected,correct_modes,chosen_mode
0,Katy makes coffee using teaspoons of sugar and...,42,"[zero_shot, few_shot, cot, pal, react]",cot
1,Who is credited with inventing the telephone?,Bell,"[zero_shot, few_shot, cot, pal, react]",few_shot
2,Where is the Eiffel Tower located?,Paris,"[zero_shot, few_shot, cot, pal, react]",react
3,Write a Python function that returns the nth F...,def,"[zero_shot, few_shot, cot, pal, react]",pal
4,What is the capital of the country whose flag ...,Ottawa,"[zero_shot, few_shot, cot, pal, react]",react
5,Which year did Alexander Graham Bell receive h...,1876,"[zero_shot, few_shot, cot, pal, react]",cot
6,Complete the function to reverse a list in Pyt...,def,"[zero_shot, few_shot, cot, pal, react]",few_shot
7,"If someone returned a lost dog to its owner, h...",grateful,"[zero_shot, few_shot, pal]",few_shot
8,Calculate the square root of 1521.,39,"[zero_shot, few_shot, cot, pal]",cot
9,"Given a CSV file of sales data, write Python c...",import,"[zero_shot, few_shot, cot, pal, react]",pal


## Feature Engineering

In [ ]:
from sklearn.preprocessing import LabelEncoder
# 2. Feature Extraction
def extract_features(query: str) -> dict:
    return {
        "length": len(query) / 100.0,
        "has_math": int(bool(re.search(r"\d+|\+|\-|\*|\/|sum|calculate", query.lower()))),
        "has_how_why": int("how" in query.lower() or "why" in query.lower()),
        "has_code_keyword": int("code" in query.lower() or "def" in query.lower() or "function" in query.lower() or "import" in query.lower()),
        "has_fact_keyword": int(bool(re.search(r"\bwho\b|\bwhat\b|\bwhere\b|\bwhen\b|\ bgdp\b|\ bcapital\b|\ btranslate\b", query.lower())))
    }

features_list = [extract_features(q) for q in df["query"]]
features_df = pd.DataFrame(features_list)  # shape: (num_rows, 5)

# 3. Encode Labels
modes = ["zero_shot", "few_shot", "cot", "pal", "react"]
label_encoder = LabelEncoder()
label_encoder.fit(modes)
y = label_encoder.transform(df["chosen_mode"])

In [107]:
features_df

,length,has_math,has_how_why,has_code_keyword,has_fact_keyword
0,1.57,1,1,0,0
1,0.45,0,0,0,1
2,0.34,0,0,0,1
3,0.62,0,0,1,0
4,0.68,0,0,0,1
5,0.74,0,0,0,0
6,0.50,0,0,1,0
7,0.70,0,1,0,0
8,0.34,1,0,0,0
9,0.79,0,0,1,0


In [108]:
y

array([0, 1, 3, 2, 3, 0, 1, 1, 0, 2, 1, 3, 0, 1, 4, 0, 2, 4, 0, 2, 4, 1,
       3, 0, 4, 1, 4, 2, 3, 0, 1, 2, 4, 0, 0, 4, 2, 4, 0])

## Reinforcement Learning

**Key Concepts:**

1. Each query is a state: We convert the question into a 5-dimensional feature vector (e.g., length, whether it has math/code/fact keywords).

2. Actions are the prompt strategies: The agent must choose between 5 actions — "zero_shot", "few_shot", "cot", "pal", or "react".

3. Reward signal: The agent receives 1 reward if it selects the correct mode (chosen_mode in your labeled data), and 0 otherwise.

4. Observation Space: The RL agent sees only the features (normalized between 0 and 1) — not the raw query text.

This environment lets the RL agent learn:
  “Given the features of a question, which prompting mode would likely return the best answer?”

In [115]:
# 0. Install WITHOUT the Box2D extra
!pip install gymnasium stable-baselines3 --quiet

import numpy as np, pandas as pd, re, gymnasium as gym
from gymnasium import spaces
from sklearn.preprocessing import LabelEncoder, MinMaxScaler     # <─ MinMax instead of StandardScaler
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# 2. Synthetic Dataset Definition
data = [
    {"query": "Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 parts, how many teaspoons of sugar did she use?", "expected": "42", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "cot"},
    {"query": "Who is credited with inventing the telephone?", "expected": "Bell", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "Where is the Eiffel Tower located?", "expected": "Paris", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Write a Python function that returns the nth Fibonacci number.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the capital of the country whose flag features a maple leaf?", "expected": "Ottawa", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Which year did Alexander Graham Bell receive his patent for the telephone?", "expected": "1876", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "cot"},
    {"query": "Complete the function to reverse a list in Python.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "If someone returned a lost dog to its owner, how would the owner feel?", "expected": "grateful", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "few_shot"},
    {"query": "Calculate the square root of 1521.", "expected": "39", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Given a CSV file of sales data, write Python code to compute the total revenue.", "expected": "import", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "In a conversation, 'Give me directions to the nearest bakery,' what is the user’s intent?", "expected": "intent", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "few_shot"},
    {"query": "According to the latest NASA press release, what is the name of their newest Mars rover?", "expected": "Perseverance", "correct_modes": ["react"], "chosen_mode": "react"},
    {"query": "Solve for x: 3x + 7 = 22.", "expected": "5", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "cot"},
    {"query": "Translate the following English sentence to French: 'The quick brown fox jumps over the lazy dog.'", "expected": "rapide", "correct_modes": ["zero_shot", "few_shot"], "chosen_mode": "few_shot"},
    {"query": "Which library in Python is most commonly used for data manipulation with DataFrames?", "expected": "pandas", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "How many minutes are there in 3.5 hours?", "expected": "210", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Write a SQL query to select all customers from New York from a 'customers' table.", "expected": "SELECT", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "pal"},
    {"query": "Who painted the Mona Lisa?", "expected": "da Vinci", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Explain why the sky appears red at sunset.", "expected": "Rayleigh", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "cot"},
    {"query": "Compute the factorial of 7 using Python.", "expected": "factorial(7)", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the boiling point of water at sea level in Celsius?", "expected": "100°C", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Who wrote 'Pride and Prejudice'?", "expected": "Austen", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "few_shot"},
    {"query": "What is the GDP of Nigeria in 2020 (in billions)?", "expected": "432", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "Solve the integral ∫ (2x³ + 3x²) dx.", "expected": "½x⁴ + x³", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Write a bash command to list all files in the current directory.", "expected": "ls", "correct_modes": ["zero_shot", "few_shot", "pal"], "chosen_mode": "zero_shot"},
    {"query": "Translate 'Good morning, how are you?' to Spanish.", "expected": "Buenos", "correct_modes": ["zero_shot", "few_shot"], "chosen_mode": "few_shot"},
    {"query": "What is the capital city of Australia?", "expected": "Canberra", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "zero_shot"},
    {"query": "Given a list of integers, write Python code to return only the even numbers.", "expected": "def", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "Which planet in our solar system has the most moons?", "expected": "Jupiter", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "react"},
    {"query": "If a car travels at 60 mph, how far will it travel in 2.5 hours?", "expected": "150", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Who is the author of '1984' and in what year was it first published?", "expected": "Orwell", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "few_shot"},
    {"query": "Create a JavaScript function to check if a string is a palindrome.", "expected": "function", "correct_modes": ["zero_shot", "few_shot", "cot", "pal", "react"], "chosen_mode": "pal"},
    {"query": "What is the chemical symbol for gold?", "expected": "Au", "correct_modes": ["zero_shot", "few_shot", "cot"], "chosen_mode": "zero_shot"},
    {"query": "Explain the difference between supervised and unsupervised learning.", "expected": "labeled", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "cot"},
    {"query": "Find the shortest path between two nodes in a graph using pseudocode.", "expected": "while", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"},
    {"query": "Who won the FIFA World Cup in 2018?", "expected": "France", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "Write Python code to generate a random integer between 1 and 100.", "expected": "import", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "pal"},
    {"query": "In Shakespeare's 'Hamlet,' who says 'To be, or not to be'?", "expected": "Hamlet", "correct_modes": ["zero_shot", "few_shot", "cot", "react"], "chosen_mode": "zero_shot"},
    {"query": "Solve for y: 2y² – 8y + 6 = 0.", "expected": "y=1 or y=3", "correct_modes": ["zero_shot", "few_shot", "cot", "pal"], "chosen_mode": "cot"}
]
df = pd.DataFrame(data)

# --------------------------------------------------------------
# 2. feature extraction
# --------------------------------------------------------------
def extract_features(q):
    return {
        "length": len(q)/100,
        "has_math": int(bool(re.search(r"\d", q))),
        "has_how_why": int("how" in q.lower() or "why" in q.lower()),
        "has_code_kw": int("def" in q.lower() or "code" in q.lower()),
        "has_fact_kw": int(bool(re.search(r"\bwho\b|\bwhat\b|\bwhere\b|\bwhen\b", q.lower())))
    }

feat_df = pd.DataFrame([extract_features(q) for q in df["query"]])

# scale to [0,1] so it really matches 0-1 box
scaler = MinMaxScaler(feature_range=(0,1))
X_scaled = scaler.fit_transform(feat_df.values)

# --------------------------------------------------------------
# 3. environment
# --------------------------------------------------------------
modes = ["zero_shot", "few_shot", "cot", "pal", "react"]
le = LabelEncoder().fit(modes)

# Turn query-routing into an RL problem
class RoutingEnv(gym.Env):
    metadata = {"render_modes": ["human"]}
    def __init__(self, df, X):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.X = X.astype(np.float32) # Feature matrix (observations).
        self.y = le.transform(df["chosen_mode"]) # Target labels (chosen_mode) converted into integers using LabelEncoder.
        self.action_space = spaces.Discrete(len(modes)) # The agent can pick one of 5 discrete actions (0 to 4), each representing a reasoning mode.
        self.observation_space = spaces.Box(
            low = 0.0, high = 1.0, shape=(5,), dtype=np.float32
        ) # Each observation is a 5-dimensional vector with values between 0 and 1.
        self.idx = None # Keeps track of the current query index.

    def reset(self, seed=None, options=None):
        #Picks a random query from the dataset.
        #Returns its feature vector (self.X[self.idx]) as the initial observation to the agent.
        self.idx = np.random.randint(len(self.df))
        return self.X[self.idx], {}


    def step(self, action):
      # The agent chooses an action (e.g., predicts "cot").
      # If it matches the correct routing mode (self.y[self.idx]), it gets a reward of 1. Otherwise, reward = 0.
      # Returns:
        # The same observation (query features) again (stateless)
        # The reward (1 or 0)
        # done = True → Each episode is 1-step long
        # truncated = False, info = {} → standard placeholders

        reward = 1 if action == self.y[self.idx] else 0
        done, truncated, info = True, False, {}
        return self.X[self.idx], reward, done, truncated, info

env = RoutingEnv(df, X_scaled)
check_env(env)      # ✅ passes now

# --------------------------------------------------------------
# 4. quick PPO train
# --------------------------------------------------------------
# Initialize PPO (Proximal Policy Optimization):
model = PPO("MlpPolicy", env, verbose=0).learn(500)

# inference helper
def route(query: str) -> str:
    # 1. feature -> scaled 1×5 array
    v = scaler.transform([list(extract_features(query).values())]).astype(np.float32)
    # 2. predict returns (actions, state)
    act_arr, _ = model.predict(v, deterministic=True)
    act_idx = int(act_arr[0])          # <-- extract scalar
    return modes[act_idx]

print(route("Who wrote Hamlet?"), "← predicted mode") # 	Use route() function	Make real-time routing decisions for new queries

few_shot ← predicted mode
